# Import needed libraries

In [1]:
import os
#import matplotlib.pyplot as plt
import re
import numpy as np
from path import Path
import csv
from pandas import *
import pandas as pd
import sys
import ast

In [2]:
from func_search import *
from func_taux_transm import *
from func_def_class import *
from func_res import *
from func_parsing_logfile import *
from func_parsing_csvfile import *

# Parse data file into specific log files

In [3]:
#### PARSING DES LOGS DANS DES SOUS FICHIER TXT
sep_data_addr("sim_1_mod")
sep_data_event("sim_1_mod")

# Create paquet transmission and reception dataframes

In [5]:
#### CREATION DES STRUCT PANDAS
#fichier tx
df_tx = create_df_tx("data/parsed/event/STAT_PK_TX.log")
df_tx.to_csv('data_csv/sim_1_mod_pkt_tx.csv',index=False)

KeyboardInterrupt: 

In [4]:
# fichier rx
df_rx = create_df_rx("data/parsed/event/STAT_PK_RX.log", "data/parsed/event/STAT_ACK.log")
df_rx.to_csv('data_csv/pkt_rx.csv',index=False)

# Complete the transmission dataframe

In [3]:
#Change some columns type for the nexts operations
df_tx = pd.read_csv(r'data_csv/pkt_tx.csv', dtype=str)
df_tx["asn"] = df_tx["asn"].astype(int)
df_tx["numTxAttempts"] = df_tx["numTxAttempts"].astype(int)
#df_tx["trackinstance"] = df_tx["trackinstance"].astype(int)
df_tx = df_tx.fillna('')
df_rx = pd.read_csv(r'data_csv/pkt_rx.csv', dtype=str)
df_rx["asn"] = df_rx["asn"].astype(int)

In [6]:
df_tx

,time,addr,asn,length,frameType,slotOffset,frequency,l2Dest,txpower,numTxAttempts,L3Src,L3Dest,L4Proto,L4SrcPort,L4DestPort,succes_rx,succes_ack,list_rx
0,"2017-07-20 15:20:29,193",0001,22,45,IEEE154_TYPE_BEACON,0,25,ffff000000000000,31,1,00000000000000000000000000000000,00000000000000000000000000000000,IANA_UNDEFINED,0,0,0,0,[]
1,"2017-07-20 15:20:29,476",0001,154,45,IEEE154_TYPE_BEACON,0,12,ffff000000000000,31,1,00000000000000000000000000000000,00000000000000000000000000000000,IANA_UNDEFINED,0,0,0,0,[]
2,"2017-07-20 15:20:29,765",0001,286,45,IEEE154_TYPE_BEACON,0,20,ffff000000000000,31,1,00000000000000000000000000000000,00000000000000000000000000000000,IANA_UNDEFINED,0,0,0,0,[]
3,"2017-07-20 15:20:30,076",0001,429,45,IEEE154_TYPE_BEACON,0,14,ffff000000000000,31,1,00000000000000000000000000000000,00000000000000000000000000000000,IANA_UNDEFINED,0,0,0,0,[]
4,"2017-07-20 15:20:30,406",0001,572,45,IEEE154_TYPE_BEACON,0,24,ffff000000000000,31,1,00000000000000000000000000000000,00000000000000000000000000000000,IANA_UNDEFINED,0,0,0,0,[]
5,"2017-07-20 15:20:30,612",0001,660,95,IEEE154_TYPE_DATA,0,26,ffffffffffffffff,31,1,ff02000000000000000000000000001a,fe80000000000000141592cc00000001,IANA_ICMPv6,155,0,0,0,[]
6,"2017-07-20 15:20:30,690",0001,693,45,IEEE154_TYPE_BEACON,0,15,ffffffffffffffff,31,1,ff02000000000000000000000000001a,fe80000000000000141592cc00000001,IANA_UNDEFINED,155,0,0,0,[]
7,"2017-07-20 15:20:31,021",0001,836,45,IEEE154_TYPE_BEACON,0,26,ffffffffffffffff,31,1,ff02000000000000000000000000001a,fe80000000000000141592cc00000001,IANA_UNDEFINED,155,0,1,0,"['0002', '000d', '000c', '000b', '0009']"
8,"2017-07-20 15:20:31,768",0001,968,45,IEEE154_TYPE_BEACON,0,19,ffffffffffffffff,31,1,ff02000000000000000000000000001a,fe80000000000000141592cc00000001,IANA_UNDEFINED,155,0,1,0,"['000b', '000d', '000c', '0002']"
9,"2017-07-20 15:20:32,471",0001,1100,45,IEEE154_TYPE_BEACON,0,24,ffffffffffffffff,31,1,ff02000000000000000000000000001a,fe80000000000000141592cc00000001,IANA_UNDEFINED,155,0,1,0,"['000b', '0002', '000d']"


In [11]:
#fill the reception columns in the tx df to know what paquet was received and ACK
df_tx = fill_succes_tx(df_tx,df_rx)
df_tx.to_csv('data_csv/pkt_tx.csv',index=False)

# Create the reservation dataframe

In [12]:
#creer la df qui contient les infos sur les res
df_res_step=create_df_step_res("data/parsed/event/data_OTHER_ParserPrintf.log")
df_res_step.to_csv('data_csv/res_step.csv',index=False)

In [7]:
# fichiers res
df_res=create_df_res("data/parsed/event/STAT_6PCMD.log").copy()
df_res.to_csv('data_csv/res.csv',index=False)

In [3]:
df_res = pd.read_csv(r'data_csv/res.csv', dtype=str)
df_res["asn"] = df_res["asn"].astype(int)
#df_res["id_res"] = df_res["id_res"].astype(int)
#df_res["succes"] = df_res["succes"].astype(int)
#df_res["state"] = df_res["state"].astype(int)
df_res = df_res.fillna('')
df_res.sort_values(['asn', 'command', 'status'], ascending=[True, False, False], inplace=True)
df_res = df_res.reset_index(drop=True)

In [4]:
df_res=fill_succes_id(df_res).copy()

error:Reservation 4 failed without failed message
error:Reservation 11 failed without failed message
error:Reservation 110 failed without failed message


In [5]:
df_res=set_state(df_res)
df_res.to_csv('data_csv/res.csv',index=False)

# create parent/son dataframe

In [6]:
#créer les fichiers parent et fils 
df_parent = create_df_parent("data/parsed/event/STAT_CELL.log")
df_parent = df_parent.drop_duplicates()
df_parent = df_parent.reset_index(drop=True)
df_parent.to_csv('data_csv/parent.csv',index=False)

In [7]:
df_parent = pd.read_csv(r'data_csv/parent.csv', dtype=str)
df_parent["asn"] = df_parent["asn"].astype(int)
df_parent = df_parent.fillna('')


In [8]:
df_fils = create_df_sons(df_parent)
df_fils.to_csv('data_csv/sons.csv',index=False)

In [9]:
df_fils = pd.read_csv(r'data_csv/sons.csv', dtype=str)
df_fils["asn"] = df_fils["asn"].astype(int)


In [10]:
df_res = pd.read_csv(r'data_csv/res.csv', dtype=str)
#df_res["numAttempts"] = df_res["numAttempts"].astype(int)
#df_res["nb_sibl"] = df_res["nb_sibl"].astype(int)
df_res["succes"] = df_res["succes"].astype(int)
df_res["asn"] = df_res["asn"].astype(int)
#df_res["state"] = df_res["state"].astype(int)
#df_res["asn creation"] = df_res["asn creation"].astype(int)
df_res = df_res.fillna('')
#df_res.sort_values(['asn creation', 'asn_req'], ascending=[True, True], inplace=True)

In [11]:
df_res = fill_nb_siblings(df_res, df_fils ,df_parent)
df_res.to_csv('data_csv/res.csv',index=False)

KeyError: 'src'

In [21]:
# a lancer après les siblings
df_res_sim = fill_simult(df_res, df_tx)
df_res_sim.to_csv('data_csv/res_sim.csv',index=False)

In [22]:
df_res = pd.read_csv(r'data_csv/res_sim.csv', dtype=str)
df_res["numAttempts"] = df_res["numAttempts"].astype(int)
df_res["nb_sibl"] = df_res["nb_sibl"].astype(int)
df_res["succes"] = df_res["succes"].astype(int)
df_res["state"] = df_res["state"].astype(int)
df_res["asn creation"] = df_res["asn creation"].astype(int)
df_res = df_res.fillna('')
df_res.sort_values(['asn creation', 'asn_req', 'asn_tx'], ascending=[True, True, True], inplace=True)